In [1]:
import os

import numpy as np
import xarray as xr

from eval_utilities import spatial_temporal_metrics as stm
from eval_utilities import visualization as vis
import matplotlib.pyplot as plt

/data/conda/envs/ailand/lib/python3.12/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


# Load Configuration

In [2]:
import yaml
with open(f"config.yaml") as stream:
    try:
        CONFIG = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [3]:
compute_metrics = True
create_visualizations = True
compute_scoreboard = False #no scoreboard yet

eval_timespan = slice("2021-01-01T00", "2022-11-30T00")

# Load Reference Data Sets

The MODIS and SMAP data need to be brought into the right format to work with our modules.

In [4]:
ds_ref = xr.open_zarr(CONFIG["path_ec_euro"])

# MODIS
ds = xr.open_zarr("/data/ch23/data_ch23/modis_temp_interp.zarr")
ds_modis = xr.DataArray(
    data = ds["modis_temp"].data[:,:,np.newaxis],
    coords = {"x":ds["x"], "time":ds["time"], "variable":["skt"]},
    dims = ["time", "x", "variable"],
    name = "data"
)
ds_modis = ds_modis.assign_coords(lon=("x", ds_ref["lon"].data))
ds_modis = ds_modis.assign_coords(lat=("x", ds_ref["lat"].data))
ds_modis = ds_modis.to_dataset()
ds_modis = ds_modis.dropna(dim="time", how="all")

# SMAP
ds = xr.open_zarr("/data/ch23/data_ch23/smap_sm_interp.zarr")
ds_smap = xr.DataArray(
    data = ds["smap_sm"].data[:,:,np.newaxis],
    coords = {"x":ds["x"], "time":ds["time"], "variable":["swvl1"]},
    dims = ["time", "x", "variable"],
    name = "data"
)
ds_smap = ds_smap.assign_coords(lon=("x", ds_ref["lon"].data))
ds_smap = ds_smap.assign_coords(lat=("x", ds_ref["lat"].data))
ds_smap = ds_smap.to_dataset()
ds_smap = ds_smap.dropna(dim="time", how="all")

/data/conda/envs/ailand/lib/python3.12/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [5]:
inf_paths = CONFIG["inf_paths"]
eval_paths = CONFIG["eval_paths"]

weights = {"swvl1": ds_ref.clim_data.sel(clim_variable="clim_theta_cap"), #use field capacity to emphasize potentially moist grid points
           "skt": None}

In [6]:
def eval_helper(func, ds_mod, path, relative=False):
    # MODIS
    dts = np.intersect1d(ds_modis["time"], ds_mod["time"])
    if "skt" in ds_mod["variable"]:
        if relative:
            result_modis = func(ds_mod.sel(time=dts), ds_modis.sel(time=dts), vars=["skt"], relative=True).to_dataset(name='data')
        else:
            result_modis = func(ds_mod.sel(time=dts), ds_modis.sel(time=dts), vars=["skt"]).to_dataset(name='data')

    # SMAP
    dts = np.intersect1d(ds_smap["time"], ds_mod["time"])
    if "swvl1" in ds_mod["variable"]:
        if relative:
            result_smap = func(ds_mod.sel(time=dts), ds_smap.sel(time=dts), vars=["swvl1"], relative=True).to_dataset(name='data')
        else:
            result_smap = func(ds_mod.sel(time=dts), ds_smap.sel(time=dts), vars=["swvl1"]).to_dataset(name='data')

    xr.concat((result_smap, result_modis), dim="variable").to_zarr(path, mode='w')

## ECLand

In [7]:
path = "/data/ch23/evalution_results/ecland/observations/"
path_vis = "/data/ch23/evalution_results/ecland/visualization/observations/"

if True:
    # Compute and store metrics:
    
    ds_mod = ds_ref
    ds_mod["variable"] = ds_mod["variable"].astype("str") #some weird xarray bug caused by "variable" (https://github.com/pydata/xarray/issues/3476)
    ds_mod = ds_mod.chunk({'time': 4, 'x': 10051, 'variable': 17})

    if not os.path.exists(path):
        os.makedirs(path)

    # Bias
    eval_helper(stm.bias, ds_mod, path + "bias.zarr", relative=False)
    
    # Bias (relative)
    eval_helper(stm.bias, ds_mod, path + "nor_bias.zarr", relative=True)

    # RMSE
    eval_helper(stm.rmse, ds_mod, path + "rmse.zarr", relative=False)

    # RMSE (relative)
    eval_helper(stm.rmse, ds_mod, path + "nor_rmse.zarr", relative=True)

    # ACC
    eval_helper(stm.acc, ds_mod, path + "acc.zarr", relative=False)


if create_visualizations:
    # Visualization:
    if not os.path.exists(path_vis):
            os.makedirs(path_vis)

    # Bias
    bias = xr.open_zarr(path + "bias.zarr")
    for var in bias.variable.values:
            vis.vis_zarr_map(bias, var, path_vis + 'bias', 1, 99)

    # Normalized bias
    nor_bias = xr.open_zarr(path + "nor_bias.zarr")
    for var in nor_bias.variable.values:
            vis.vis_zarr_map(nor_bias, var, path_vis + 'nor_bias', 1, 99)  

    # RMSE
    rmse = xr.open_zarr(path + "rmse.zarr")
    for var in rmse.variable.values:
            vis.vis_zarr_map(rmse, var, path_vis + 'rmse', 1, 99)

    # Normalized RMSE
    nor_rmse = xr.open_zarr(path + "nor_rmse.zarr")
    for var in nor_rmse.variable.values:
            vis.vis_zarr_map(nor_rmse, var, path_vis + 'nor_rmse', 1, 99)  

    # ACC
    acc = xr.open_zarr(path + "acc.zarr")
    for var in acc.variable.values:
            vis.vis_zarr_map(acc, var, path_vis + 'acc', 1, 99)


/data/conda/envs/ailand/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/data/conda/envs/ailand/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/data/conda/envs/ailand/lib/python3.12/site-packages/dask/array/numpy_compat.py:53: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
/data/conda/envs/ailand/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


/data/conda/envs/ailand/lib/python3.12/site-packages/dask/array/core.py:4832: PerformanceWarning: Increasing number of chunks by factor of 16
  result = blockwise(
/data/conda/envs/ailand/lib/python3.12/site-packages/dask/array/core.py:4832: PerformanceWarning: Increasing number of chunks by factor of 16
  result = blockwise(
/data/conda/envs/ailand/lib/python3.12/site-packages/dask/array/core.py:4832: PerformanceWarning: Increasing number of chunks by factor of 16
  result = blockwise(
/data/conda/envs/ailand/lib/python3.12/site-packages/dask/array/core.py:4832: PerformanceWarning: Increasing number of chunks by factor of 16
  result = blockwise(
/data/conda/envs/ailand/lib/python3.12/site-packages/dask/array/core.py:4832: PerformanceWarning: Increasing number of chunks by factor of 16
  result = blockwise(
/data/conda/envs/ailand/lib/python3.12/site-packages/dask/array/core.py:4832: PerformanceWarning: Increasing number of chunks by factor of 16
  result = blockwise(


## Emulators

In [8]:
for model in ["ecland"] + list(inf_paths.keys()):    
    path = eval_paths[model] + '/observations/'
    path_vis = eval_paths[model] + '/visualization/observations/'

    if compute_metrics:
        # Compute and store metrics:

        ds_mod = xr.open_zarr(inf_paths[model])
        ds_mod = ds_mod.chunk({'time': 4, 'x': 10051, 'variable': 17})
            
        if not os.path.exists(path):
            os.makedirs(path)

        # Bias
        eval_helper(stm.bias, ds_mod, path + "bias.zarr", relative=False)
        
        # Bias (relative)
        eval_helper(stm.bias, ds_mod, path + "nor_bias.zarr", relative=True)

        # RMSE
        eval_helper(stm.rmse, ds_mod, path + "rmse.zarr", relative=False)

        # RMSE (relative)
        eval_helper(stm.rmse, ds_mod, path + "nor_rmse.zarr", relative=True)

        # ACC
        eval_helper(stm.acc, ds_mod, path + "acc.zarr", relative=False)

        
    if create_visualizations:
        # Visualization:
        if not os.path.exists(path_vis):
            os.makedirs(path_vis)
        
        # Bias
        bias = xr.open_zarr(path + "bias.zarr")
        for var in bias.variable.values:
            vis.vis_zarr_map(bias, var, path_vis + 'bias', 1, 99)
        
        # Normalized bias
        nor_bias = xr.open_zarr(path + "nor_bias.zarr")
        for var in nor_bias.variable.values:
            vis.vis_zarr_map(nor_bias, var, path_vis + 'nor_bias', 1, 99)  
        
        # RMSE
        rmse = xr.open_zarr(path + "rmse.zarr")
        for var in rmse.variable.values:
            vis.vis_zarr_map(rmse, var, path_vis + 'rmse', 1, 99)

        # Normalized RMSE
        nor_rmse = xr.open_zarr(path + "nor_rmse.zarr")
        for var in nor_rmse.variable.values:
            vis.vis_zarr_map(nor_rmse, var, path_vis + 'nor_rmse', 1, 99)  

        # ACC
        acc = xr.open_zarr(path + "acc.zarr")
        for var in acc.variable.values:
            vis.vis_zarr_map(acc, var, path_vis + 'acc', 1, 99)

KeyError: 'ecland'